# Método 1

In [4]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def select_features_method1(train_values, train_labels, k_size):
  #transformo los datos
  scaler = MinMaxScaler()
  scaler.fit(train_values)
  x_train = scaler.transform(train_values)

  #entreno el método 1
  selector = SelectKBest(chi2, k_size)
  selector.fit(x_train, train_labels)

  x_train_fs = selector.transform(x_train)
  k_features = np.array(selector.get_support()).reshape((1,-1))

  selected_features = []
  features = train_values.columns.tolist()
  for i, feature in enumerate(features):
    if k_features[0][i]:
      selected_features.append(feature)

  return selected_features

# Método 4

In [7]:
from xgboost import XGBClassifier

def insert_feature(selected_features_to_return, feature, verbose):
  if verbose:
    selected_features_to_return.append(feature)
  else:
    selected_features_to_return.append(feature[0])

  return selected_features_to_return

def select_features_method4(train_values, train_labels, k_size, k_importance=False, verbose=False):
  scaler = MinMaxScaler()
  scaler.fit(train_values)
  x_train = scaler.transform(train_values)

  model = XGBClassifier(
    max_depth=8,
    n_estimators=100,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

  model.fit(x_train, train_labels.values.ravel(), verbose=True)

  importances = model.feature_importances_
  features = train_values.columns.tolist()
  ordered_features = {}

  for i, feature in enumerate(features): 
    ordered_features.update({feature: importances[i]})

  selected_features = sorted(ordered_features.items(), key=lambda item: item[1], reverse=True)

  selected_features_to_return = []
  
  for i, feature in enumerate(selected_features):
    if k_importance:
      if feature[1] >= k_size:
        selected_features_to_return = insert_feature(selected_features_to_return, feature, verbose)

    else:
      if i < k_size:
        selected_features_to_return = insert_feature(selected_features_to_return, feature, verbose)

  return selected_features_to_return

# Ejemplo de uso

In [19]:
#index = train_values.index.tolist()

In [ ]:
#selected_features = select_features_method4(train_values.loc[index[0:20000]], train_labels.loc[index[0:20000]], 0.0089, True, True)
#selected_features